<a href="https://colab.research.google.com/github/Hevej/Machine-Learning/blob/main/ManualMateriales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
import pandas as pd
import numpy as np

# Ingresar los datos



## Ingresar los datos desde el Drive

Primero se establece la conexión en el Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Despues se debe cambiar la ruta en la variable ``` filename ``` por la ruta donde se encuentre el archivo



In [ ]:
filename = '/content/drive/MyDrive/Dataset_Materiales/BrazoRobotico.xlsx'

In [ ]:
data = pd.read_excel(filename, sheet_name ='Materiales')

## Ingresar los datos desde memoria

Se debe arrastrar el archivo al colab

Despues se debe cambiar la ruta en la variable ``` filename ``` por la ruta donde se encuentre el archivo



In [ ]:
filename = '/content/BrazoRobotico.xlsx'

In [ ]:
data = pd.read_excel(filename, sheet_name ='Materiales')

# Ingresar los datos para el método AHP
Estas lineas solo se deben correr si se tienen los datos dentro del EXCEL correspondientes a los requeridos por método AHP

In [ ]:
criteria = pd.read_excel(filename, sheet_name ='Criterios', dtype = float, usecols=lambda x: 'Unnamed' not in x)

In [ ]:
properties = dict()
for i in range(len(data.index)):
  properties[data['In'][i]] = pd.read_excel(filename, sheet_name = 'C{0}'.format(i+1), dtype = float, usecols=lambda x: 'Unnamed' not in x)

# Cargar la clase con sus metodos

In [ ]:
class MaterialSelection():
  def __init__(self, DB):
    self.DB = DB
    self.propertiesVector = DB['No.']
    self.materialsVector = DB.columns[5:]
    self.intention = DB[['No.','Intención']].set_index('No.')

  def __cleanVsMatrix(self):
    return pd.DataFrame(0.0, index=self.materialsVector, columns=self.materialsVector)

  ###################
  ######  AHP  ######
  ###################

  def __iterConsistency(self, matrix):
    normalSum = matrix.sum(axis=1)
    normalTotal = normalSum.sum()
    normalizeSum = normalSum/normalTotal
    return normalizeSum

  #Metodo que calcula el vector lambda
  def __lambda(self, matrix):
    # matrix es la matriz pareada
    matrix = matrix.to_numpy()
    #Genera por lo menos una vez, Do-While
    matrix =  matrix@matrix
    anterior = self.__iterConsistency(matrix)
    while True:
      matrix = matrix@matrix
      normalizeSum = self.__iterConsistency(matrix)
      #Se sale cuando el error esta por debajo de 1e-5
      if np.max(normalizeSum - anterior) <= 0.00001:
        break
      else:
        anterior = normalizeSum
    #Retorna el vector lambda
    return normalizeSum

  #Calcula los p valores de la matriz pareada
  def __pValues(self, matrix):
    return matrix.sum(axis=0)

  def __aleatoryConsistencyIndex(self, n):
    rIValue = np.array([0, 0, 0.58, 0.89, 1.12, 1.26, 1.36, 1.41, 1.42, 1.49, 1.54, 1.56, 1.58])
    if n<= 14:
      return rIValue[n-1]
    else:
      return rIValue[-1]

  def __consistency(self, matrix):
    lambdaVector = self.__lambda(matrix)
    lambdaMax = np.dot(lambdaVector, self.__pValues(matrix))
    #Número de críterios
    n = len(matrix.index)
    #Índice de consistencia
    CI = (lambdaMax - n)/(n-1)
    #Índice aleatorio de consistencia
    RI = self.__aleatoryConsistencyIndex(n)
    CR = CI/RI
    return '{0}%'.format(np.around(CR*100, decimals = 2)) , lambdaVector

  def __clasificationAHP(self, arguments):
    weightedScore = []
    sol = {'Alternativa': self.materialsVector}
    for i in self.propertiesVector:
      sol[i] = arguments[i]
    sol = pd.DataFrame(data=sol).set_index('Alternativa')
    for i in range(len(self.materialsVector)):
      weightedScore.append(np.dot(sol.iloc[i],arguments['Pesos']))
    sol['Puntaje Ponderado'] = weightedScore
    sol['Clasificación'] = sol['Puntaje Ponderado'].rank(ascending = False)
    return sol

  def mAHP(self, criterios, arrays=0, pesos=True):
    arguments = dict()
    consistentRatios = []
    i = 1
    #Matriz de consistencia para las propiedades
    a, b = self.__consistency(criterios)
    arguments['Pesos'] = b
    if pesos:
      consistentRatios.append(a)
      arguments['CR'] = consistentRatios
      return arguments
    else:
      #Matriz de consistencia para los materiales
      for matrix in arrays.values():
        a, b = self.__consistency(matrix)
        consistentRatios.append(a)
        arguments['C{0}'.format(i)] = b
        i += 1
      arguments['CR'] = consistentRatios
      sol = self.__clasificationAHP(arguments)
      #Aca va la función de clasificación
      return arguments, sol

  ###################
  ####  ELECTRE  ####
  ###################

  def __decisionalMatrix(self):
    decsMatrix = dict()
    decsMatrix = {'Alternativa': self.propertiesVector.to_numpy()}
    for i in range(len(self.materialsVector)):
      decsMatrix[self.materialsVector[i]] =  self.DB[self.materialsVector[i]]
    return pd.DataFrame.from_dict(decsMatrix).set_index('Alternativa').transpose()
    
  def __normalize(self):
    decisional = self.__decisionalMatrix()
    sumRij2 = []
    for col in decisional:
      if self.intention['Intención'].loc[col] == 'Max':
        decisional[col] = decisional[col]/np.sqrt((decisional[col]**2).sum())
      else:
        decisional[col] = (1/decisional[col])/np.sqrt((1/(decisional[col]**2)).sum())
    return decisional

  def __weights(self, weights, criteria, ahp):
    if ahp:
      return self.mAHP(criteria, pesos=True)['Pesos']
    else:
      return weights

  def __weightedMatrix(self, decisional, weights):
    i = 0
    for col in decisional:
      decisional[col] = decisional[col]*weights[i]
      i += 1
    return decisional
  
  def __weightedComparison(self, vSup, vInf, weights):
    totalWight = 0
    for i in range(len(vSup)):
      if vSup[i] >= vInf[i]:
          totalWight += weights[i]
    return totalWight

  def __weightedDifference(self, vSup, vInf, index, col, params):
    dif = vSup - vInf
    MDN = np.min(dif)
    MDT = np.max(np.abs(dif))
    params['{0}-{1}'.format(index, col)] = np.concatenate((dif, MDN, MDT), axis=None)
    return params

  def __concordanceMatrix(self, wMatrix, weights):
    #Inicializar la matriz
    params = {}
    cMatrix = self.__cleanVsMatrix()
    for index, row in cMatrix.iterrows():
      for column, value in row.iteritems():
        if index == column:
          continue
        else:
          cMatrix[column].loc[index] = self.__weightedComparison(wMatrix.loc[index], wMatrix.loc[column], weights)
          params = self.__weightedDifference(wMatrix.loc[index], wMatrix.loc[column], index, column, params)
    diffMatrix = pd.DataFrame(params, index=np.concatenate((self.propertiesVector, 'MDN', 'MDT'), axis=None)).transpose()
    return cMatrix, diffMatrix

  def __discordanceMatrix(self, diffMatrix):
    #Inicializar la matriz
    dMatrix = self.__cleanVsMatrix()
    for index, row in dMatrix.iterrows():
      for column, value in row.iteritems():
        if index == column:
          continue
        else:
          diffIndex = '{0}-{1}'.format(index, column)
          dMatrix[column].loc[index] = np.abs(diffMatrix['MDN'].loc[diffIndex])/np.abs(diffMatrix['MDT'].loc[diffIndex])
    return dMatrix

  #Umbral de concordancia y discordancia
  def __valuesUCUD(self, c, d):
    nc = c.size
    nd = d.size
    UC = (c.to_numpy().sum())/(nc-np.sqrt(nc))
    UD = (d.to_numpy().sum())/(nd-np.sqrt(nd))
    return UC, UD

  #CC Dominante y DC Dominante
  def __ccdcDominant(self, cc, uc, dc, ud):
    ccD = cc.applymap(lambda x: 1 if x>=uc else 0)
    dcD = dc.applymap(lambda x: 1 if x<=ud else 0)
    return ccD, dcD

  def __clasificationELECTRE(self, inf, sup):
    sol = {'Dominancia':sup-inf}
    sol['Clasificación'] = np.floor(sol['Dominancia'].rank(ascending = False))
    return pd.DataFrame(sol, index = self.materialsVector)

  def mELECTRE(self, weights = 0, criteria = 0, ahp=False):
    weights = self.__weights(weights, criteria, ahp)
    weightedMatrix = self.__weightedMatrix(self.__normalize(), weights)
    concordanceMatrix, diffMatrix = self.__concordanceMatrix(weightedMatrix, weights)
    discordanceMatrix = self.__discordanceMatrix(diffMatrix)
    UC, UD = self.__valuesUCUD(concordanceMatrix, discordanceMatrix)
    ccDominant, dcDominant = self.__ccdcDominant(concordanceMatrix, UC, discordanceMatrix, UD)
    aggregateDominance = ccDominant.multiply(dcDominant)
    inferior, superior = aggregateDominance.sum(axis=0), aggregateDominance.sum(axis=1)
    sol = self.__clasificationELECTRE(inferior, superior)
    return sol

  ###################
  ###  PROMETHEE  ###
  ###################

  def __functions(self, name, data, q, p):
    if name=='I':
      return data.applymap(lambda x: 1 if x> 0 else 0)
    elif name =='II':
      return data.applymap(lambda x: 1 if x>q else 0)
    elif name =='III':
      return data.applymap(lambda x: 1 if x>p else (x/p if x>0 else 0))
    elif name =='IV':
      return data.applymap(lambda x: 1 if x>p else (1/2 if x>q else 0))
    elif name == 'V':
      return data.applymap(lambda x: 1 if x>p else ((x/(p-q))+(q/(q-p)) if x>q else 0))
    elif name == 'VI':
      positives = data.applymap(lambda x: np.abs(x))
      n = data.size
      s = positives.stack().sum()/(n-np.sqrt(n))
      return data.applymap(lambda x: 1-np.exp(-(x**2)/(2*(s**2))) if x> 0 else 0 )

  def __diffMatrix(self, criterio, proper):
    decisional = self.__decisionalMatrix()
    tempMatrix = self.__cleanVsMatrix()
    i = 0
    for column in tempMatrix:
      if criterio == 'Max':
        tempMatrix[column] = decisional[proper] - decisional[proper].iloc[i]
      else:
        tempMatrix[column] = -decisional[proper] + decisional[proper].iloc[i]
      i += 1
    return pd.DataFrame(tempMatrix)

  def __diffArrys(self, w):
    tempMatrix = dict()
    for index, row in self.intention.iterrows():
      tempMatrix[index] = self.__diffMatrix(row[0], index)
    return tempMatrix

  def __alternativeMatrix(self, data, functions, q, p):
    i = 0
    for key in data:
      data[key] = self.__functions(functions[i], data[key], q[i], p[i])
      i += 1
    return data

  def __arrayPreferenceIndices(self, data, weights):
    i = 0
    total = self.__cleanVsMatrix()
    for key in data:
      total = data[key]*weights[i] + total
      i += 1
    return total

  def __fs(self, data):
    sol = data.copy()
    n = len(data)-1
    sol['FS+'] = data.sum(axis=1)/n
    sol['FS-'] = data.sum(axis=0)/n
    return sol[['FS+','FS-']]
  
  def __clasificationPROMETHEE(self, fsPositive, fsNegative):
    sol = {'FS_N': fsPositive - fsNegative}
    sol['Clasificación'] = sol['FS_N'].rank(ascending = False)
    return pd.DataFrame(sol, index = self.materialsVector)

  def mPROMETHEE(self, functions, q, p, weights = 0, diff = False,  criteria = 0, ahp=False):
    weights = self.__weights(weights, criteria, ahp)
    diffArrays = self.__diffArrys(weights)
    if diff:
      return diffArrays
    alternativeMatrix = self.__alternativeMatrix(diffArrays, functions, q, p)
    arrayPreferenceIndices = self.__arrayPreferenceIndices(alternativeMatrix, weights)
    fs = self.__fs(arrayPreferenceIndices)
    sol = self.__clasificationPROMETHEE(fs['FS+'], fs['FS-'])
    return sol

  ###################
  #####  TOPSIS #####
  ###################

  def __rij2(self, decisional):
    rij2 = decisional.applymap(lambda x: x**2)
    return rij2, rij2.sum(axis=0)
  
  def __normalizedRij2(self, decisional, rij2, sum):
    i = 0
    for col in rij2:
      rij2[col] = decisional[col]/np.sqrt(sum[i])
      i += 1
    return rij2

  def __v(self, wMatrix, letter):
    vPositive = []
    vNegative = []
    i = 0
    for intention in self.intention.to_numpy():
      if intention == 'Max':
        vPositive.append(wMatrix.max(axis=0)[i])
        vNegative.append(wMatrix.min(axis=0)[i])
      else:
        vPositive.append(wMatrix.min(axis=0)[i])
        vNegative.append(wMatrix.max(axis=0)[i])
      i += 1
    sol = {'{0}+'.format(letter): vPositive, '{0}-'.format(letter): vNegative}
    return pd.DataFrame(sol, index = self.propertiesVector)

  def __distance(self, wMatrix,  vP, vM):
    dP, dM = [], []
    for index, row in wMatrix.iterrows():
      dP.append(np.sqrt(((row - vP)**2).sum()))
      dM.append(np.sqrt(((row - vM)**2).sum()))
    sol = {'d+': dP, 'd-': dM}
    return pd.DataFrame(sol, index=self.materialsVector)
  
  def __clasificationTOPSIS(self, dP, dN):
    sol = {'P': dN/(dN+dP)}
    sol['Clasificación'] = sol['P'].rank(ascending = False)
    return pd.DataFrame(sol, index = self.materialsVector)

  def mTOPSIS(self, weights = 0, criteria =  0, ahp = False):
    weights = self.__weights(weights, criteria, ahp)
    decisional = self.__decisionalMatrix()
    rij2, sum = self.__rij2(decisional)
    normalizedRij2 = self.__normalizedRij2(decisional, rij2, sum)
    weightedMatrix = self.__weightedMatrix(normalizedRij2, weights)
    v = self.__v(weightedMatrix, 'V')
    distances = self.__distance(weightedMatrix, v['V+'], v['V-'])
    sol = self.__clasificationTOPSIS(distances['d+'], distances['d-'])
    return sol

  ###################
  #####  VIKOR   ####
  ###################

  def __sr(self, decisional, fP, fM, weights):
    i = 0
    for col in decisional:
      decisional[col] = weights[i]*(fP[i]-decisional[col])/(fP[i]-fM[i])
      i += 1
    return decisional

  def __siRi(self, sr):
    si = sr.sum(axis=1)
    Ri = sr.max(axis=1)
    sa, Ra = np.min(si), np.min(Ri)
    sm, Rm = np.max(si), np.max(Ri)
    return sa, Ra, sm, Rm, si, Ri

  def __clasificationVIKOR(self, s_r, v):
    sa, Ra, sm, Rm, si, Ri = self.__siRi(s_r)
    sol = {'Q_i': v*(si-sa)/(sm-sa)+(1-v)*(Ri-Ra)/(Rm-Ra)}
    sol['Clasificación'] = sol['Q_i'].rank(ascending = True)
    return pd.DataFrame(sol, index = self.materialsVector)

  def mVIKOR(self, weights = 0, criteria=0, ahp = False, v=0.5):
    weights = self.__weights(weights, criteria, ahp)
    decisional = self.__decisionalMatrix()
    f = self.__v(decisional, 'f')
    s_r = self.__sr(decisional, f['f+'], f['f-'], weights)    
    sol = self.__clasificationVIKOR(s_r, v)
    return sol


# Función entropia

La siguiente función calcula los pesos por el método de entropia

In [ ]:
def entropy(data):
  materials = data.columns[5:].values.tolist()
  matrixE = data.set_index('No.')[data.columns[5:].values.tolist()]
  sumaE = matrixE.sum(axis=1)
  matrixE = matrixE.transpose()
  mE = len(data.index)
  hE = -1/np.log(mE)
  for col in matrixE:
    matrixE[col] = matrixE[col]/sumaE.loc[col]
    matrixE[col] = hE*matrixE[col]*np.log(matrixE[col])
  ejE = matrixE.sum().transpose()
  djE = 1-ejE
  sumdjE = djE.sum()
  wjE = djE/sumdjE
  return wjE

# Selección de materiales

Primero se debe crear un objeto de la clase ``` MaterialSelection```, pasandole como único argumento el Pandas DataFrame creado al leer el excel.

In [ ]:
Seleccion = MaterialSelection(data)

## Método AHP (Pesos y Clasificación)

Si queremos averiguar solo el porcentaje de CR de la matiz de criterios y los pesos, debemos correr el meétodo como se ve a continuación, en donde:
* El primer argumento es la matriz de criterios
* El segundo argumento es una bandera ``` pesos ``` en ``` True ```, que significa que no clasificaremos los materiales.
* El método devuelve el %CR y un array de pesos.

**Este método se usa de esta manera unicamente si queremos corroborar los pesos y el %CR**

In [ ]:
print(Seleccion.mAHP(criteria, pesos=True))

{'Pesos': array([0.41816103, 0.27070697, 0.12049985, 0.19063214]), 'CR': ['2.66%']}


Clasificación AHP

Acá si se generan los pesos internamente y clasifica los materiales. Devuelve dos matrices:
* Una donde se encuentra el puntaje por material y criterio, los %CR para cada criterio (C_n) y los pesos obtenidos
* La matriz de clasificación de los materiales 

In [ ]:
datosAHP, clasificacionAHP = Seleccion.mAHP(criteria, properties, pesos=False)

In [ ]:
datosAHP

{'C1': array([0.12195719, 0.31961826, 0.55842454]),
 'C2': array([0.16919987, 0.44342911, 0.38737101]),
 'C3': array([0.31961826, 0.12195719, 0.55842454]),
 'C4': array([0.63698557, 0.25828499, 0.10472943]),
 'CR': ['1.58%', '1.58%', '1.58%', '3.32%'],
 'Pesos': array([0.41816103, 0.27070697, 0.12049985, 0.19063214])}

In [ ]:
clasificacionAHP

,C1,C2,C3,C4,Puntaje Ponderado,Clasificación
Alternativa,,,,,,
Madera,0.121957,0.169200,0.319618,0.636986,0.256745,3.0
Plástico,0.319618,0.443429,0.121957,0.258285,0.317625,2.0
Metal,0.558425,0.387371,0.558425,0.104729,0.425630,1.0


## Método ELECTRE (Normal y con pesos AHP -Híbrido-)

### Con pesos de AHP
Si se quiere usar los pesos de AHP, se debe traer la matriz de criterios, los argumentos que entran al método son:
* ``` criteria ``` en donde se ingresa la matriz **criteria**, al ingresarla en la sección de entrada de datos, pertenece a la matriz de datos para calcular los pesos con AHP.
* ``` ahp ``` en ``` True ```, para asegurar que el método se ejecutara con pesos de ahp

In [ ]:
Seleccion.mELECTRE(criteria = criteria, ahp = True)

,Dominancia,Clasificación
Madera,-1,2.0
Plástico,-1,2.0
Metal,2,1.0


### Con pesos definidos por el usuario

Se ingresan primero los pesos como un ``` np.array ```

In [ ]:
pesos = np.array([0.1, 0.15, 0.3, 0.1, 0.35])

La entradas al método son:
* Los pesos, previamente definidos.



In [ ]:
Seleccion.mELECTRE(pesos)

,Dominancia,Clasificación
Madera,-1,2.0
Plástico,-1,2.0
Metal,2,1.0


## Método PROMETHEE (Normal y con pesos AHP -Híbrido-)

Primero se deben definir tres listas:
* Funciones : en donde se estableceran el tipo de funciones a usar, en números romanos.
* q : los parametros q para esas funciones, de no tener, se debe colocar un cero o número negativo.
* p : los parametros p para esas funciones, de no tener, se debe colocar un cero o número negativo.

In [ ]:
funciones = ['II', 'IV', 'V', 'III', 'VI']
q = [ 10, 1, 0.03, 0, 0]
p = [ 0, 2, 0.25, 1.8, 0]

Si no se sabe a priori que funciones o datos q y p usar, se puede el método PROMETHEE de la siguiente manera para ver el comportamiento de cada matriz:

### Matriz de diferenias

In [ ]:
output = Seleccion.mPROMETHEE(0, 0, 0, diff=True)

In [ ]:
for key in output:
  print("Matriz", key)
  print(output[key])
  print()

Matriz C1
          Madera  Plástico  Metal
Madera       0.0     -50.0 -130.0
Plástico    50.0       0.0  -80.0
Metal      130.0      80.0    0.0

Matriz C2
          Madera  Plástico   Metal
Madera       0.0     500.0  7300.0
Plástico  -500.0       0.0  6800.0
Metal    -7300.0   -6800.0     0.0

Matriz C3
          Madera  Plástico  Metal
Madera       0.0      -4.0 -199.0
Plástico     4.0       0.0 -195.0
Metal      199.0     195.0    0.0

Matriz C4
          Madera  Plástico  Metal
Madera       0.0       0.2    0.5
Plástico    -0.2       0.0    0.3
Metal       -0.5      -0.3    0.0



### Con pesos de AHP
Si se quiere usar los pesos de AHP, se debe traer la matriz de criterios, los argumentos que entran al método son:
* ``` criteria ``` en donde se ingresa la matriz **criteria**, al ingresarla en la sección de entrada de datos, pertenece a la matriz de datos para calcular los pesos con AHP.
* ``` ahp ``` en ``` True ```, para asegurar que el método se ejecutara con pesos de ahp

In [ ]:
Seleccion.mPROMETHEE(funciones, q, p, criteria = criteria, ahp = True)

,FS_N,Clasificación
Madera,-0.230887,3.0
Plástico,0.005295,2.0
Metal,0.225591,1.0


### Con pesos definidos por el usuario

Se ingresan primero los pesos como un ``` np.array ```

In [ ]:
pesos = np.array([0.1, 0.15, 0.3, 0.1, 0.35])

La entradas al método son:
* Las listas ``` funciones, q, p``` previamente definidas.
* Los pesos, previamente definidos.

In [ ]:
Seleccion.mPROMETHEE(funciones, q, p, pesos)

,FS_N,Clasificación
Madera,-0.230556,3.0
Plástico,0.002778,2.0
Metal,0.227778,1.0


## Método TOPSIS (Normal y con pesos AHP -Híbrido-)

### Con pesos de AHP
Si se quiere usar los pesos de AHP, se debe traer la matriz de criterios, los argumentos que entran al método son:
* ``` criteria ``` en donde se ingresa la matriz **criteria**, al ingresarla en la sección de entrada de datos, pertenece a la matriz de datos para calcular los pesos con AHP.
* ``` ahp ``` en ``` True ```, para asegurar que el método se ejecutara con pesos de ahp

In [ ]:
Seleccion.mTOPSIS(criteria = criteria, ahp = True)

,P,Clasificación
Madera,0.429010,3.0
Plástico,0.533529,2.0
Metal,0.570990,1.0


### Con pesos definidos por el usuario

Se ingresan primero los pesos como un ``` np.array ```

In [ ]:
pesos = np.array([0.1, 0.15, 0.3, 0.1, 0.35])

La entradas al método son:
* Los pesos, previamente definidos.

In [ ]:
Seleccion.mTOPSIS(pesos)

,P,Clasificación
Madera,0.318350,2.0
Plástico,0.312509,3.0
Metal,0.681650,1.0


## Método VIKOR (Normal y con pesos AHP -Híbrido-)

### Con pesos de AHP
Si se quiere usar los pesos de AHP, se debe traer la matriz de criterios, los argumentos que entran al método son:
* ``` criteria ``` en donde se ingresa la matriz **criteria**, al ingresarla en la sección de entrada de datos, pertenece a la matriz de datos para calcular los pesos con AHP.
* ``` ahp ``` en ``` True ```, para asegurar que el método se ejecutara con pesos de ahp

In [ ]:
Seleccion.mVIKOR(criteria = criteria, ahp = True)

,Q_i,Clasificación
Madera,1.000000,3.0
Plástico,0.057312,2.0
Metal,0.041587,1.0


### Con pesos definidos por el usuario

Se ingresan primero los pesos como un ``` np.array ```

In [ ]:
pesos = np.array([0.1, 0.15, 0.3, 0.1, 0.35])

La entradas al método son:
* Los pesos, previamente definidos.

In [ ]:
Seleccion.mVIKOR(pesos)

,Q_i,Clasificación
Madera,0.981441,3.0
Plástico,0.979899,2.0
Metal,0.000000,1.0


# Selección Entropia

In [ ]:
pesosEntropia = entropy(data)

## ELECTREE

In [ ]:
Seleccion.mELECTRE(pesosEntropia)

,Dominancia,Clasificación
Madera,-1,2.0
Plástico,-1,2.0
Metal,2,1.0


## PROMETHEE

In [ ]:
funciones = ['II', 'IV', 'V', 'III', 'VI']
q = [ 10, 1, 0.03, 0, 0]
p = [ 0, 2, 0.25, 1.8, 0]

In [ ]:
Seleccion.mPROMETHEE(funciones, q, p, pesos)

,FS_N,Clasificación
Madera,-0.230556,3.0
Plástico,0.002778,2.0
Metal,0.227778,1.0


## TOPSIS

In [ ]:
Seleccion.mTOPSIS(pesosEntropia)

,P,Clasificación
Madera,0.377673,2.0
Plástico,0.376442,3.0
Metal,0.622327,1.0


## VIKOR

In [ ]:
Seleccion.mVIKOR(pesosEntropia)

,Q_i,Clasificación
Madera,1.000000,3.0
Plástico,0.936296,2.0
Metal,0.000000,1.0
